In [1]:
%run ../../notebooks/__init__.ipynb

prody not found; if needed, install with: conda install conda-forge::prody


In [2]:
def repairpdb_fin_():
    struct_pref, struct_id = snakemake.io.glob_wildcards(pfile(struct_id='{}', step='af2.trim_bf.repairpdb', suffix='.pdb', base=''))
    #return list(filter(lambda s: not s.endswith('_sstat'), struct_id))
    return struct_id
l_repairpdb = repairpdb_fin_()
printlen(l_repairpdb)

5,953


In [3]:
def pssm_fin_():
    struct_pref, struct_id = snakemake.io.glob_wildcards(pfile(struct_id='{}', step='af2.trim_bf.repairpdb.pssm', suffix='.tsv', base=''))
    return list(filter(lambda s: not s.endswith('_sstat'), struct_id))
l_pssm = pssm_fin_()
#l_pssm = ['A0A023PZF2', 'A0A0B7P3V8']
printlen(l_pssm)

5,953


In [4]:
#df_pssm = pd.concat(map(lambda struct_id: pd.read_csv(pfile(struct_id=struct_id, step='af2.trim_bf.repairpdb.pssm', suffix='.tsv', base=''), sep='\t'), l_pssm), axis=0)
def fn_(struct_id): 
    return pd.read_csv(pfile(struct_id=struct_id, step='af2.trim_bf.repairpdb.pssm', suffix='.tsv', base=''), sep='\t')
df_pssm = pd.concat(tqdm.contrib.concurrent.process_map(fn_, l_pssm, max_workers=int(os.environ['SLURM_NTASKS']), chunksize=10), axis=0)

cols_empty = []
for col in df_pssm.columns:
    if df_pssm[col].nunique() == 1:
        print(col)
        cols_empty.append(col)

df_pssm = df_pssm.drop(['aa_ref', 'aa_pos', 'aa_alt', 'pssm_pos', 'Pdb'] + cols_empty, axis=1)
df_pssm

  0%|          | 0/5953 [00:00<?, ?it/s]

chain
SD
sloop_entropy
mloop_entropy
water bridge
electrostatic kon
partial covalent bonds
Entropy Complex


,variant_id,total energy,Backbone Hbond,Sidechain Hbond,Van der Waals,Electrostatics,Solvation Polar,Solvation Hydrophobic,Van der Waals clashes,entropy sidechain,entropy mainchain,cis_bond,torsional clash,backbone clash,helix dipole,disulfide,energy Ionisation
0,P31688/R13A,1.079610,1.84439,1.40530,1.849950,0.347089,-3.106470,1.433230,-7.302750e-03,-1.946900,-0.754708,0.0,-1.985080e-02,-0.251124,0.034871,0.0,0.000000e+00
1,P31688/R13C,1.504660,1.66476,1.65176,1.561450,0.470368,-2.819260,1.158290,-1.791460e-03,-1.650040,-0.546371,0.0,-1.750860e-02,-0.153265,0.033011,0.0,-1.776360e-15
2,P31688/R13D,1.864650,1.68617,1.65176,1.723350,0.770899,-2.903490,1.326490,-7.730210e-03,-1.858510,-0.548361,0.0,-1.764570e-02,-0.177927,0.041717,0.0,-2.664540e-15
3,P31688/R13E,1.559970,1.78584,1.39831,1.399810,0.601000,-2.370210,0.939066,-5.089890e-03,-1.646900,-0.561810,0.0,-1.837270e-02,-0.127188,0.038325,0.0,-8.881780e-16
4,P31688/R13F,0.990531,1.74064,1.41902,0.988357,0.453385,-1.885150,0.421027,5.755110e-03,-1.569650,-0.623004,0.0,9.977990e-03,0.050561,0.030168,0.0,-2.664540e-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,Q45U18/Y67S,0.282357,0.00000,0.00000,0.215280,0.007622,-0.076505,0.399265,0.000000e+00,-0.037849,-0.225456,0.0,0.000000e+00,0.000000,0.000000,0.0,0.000000e+00
1176,Q45U18/Y67T,0.152663,0.00000,0.00000,0.243062,0.007622,-0.211840,0.367165,0.000000e+00,-0.037849,-0.215496,0.0,-1.110220e-16,0.000000,0.000000,0.0,0.000000e+00
1177,Q45U18/Y67V,0.180510,0.00000,0.00000,0.199468,0.007622,-0.150159,0.313742,0.000000e+00,-0.037849,-0.152314,0.0,-3.330670e-16,0.000000,0.000000,0.0,0.000000e+00
1178,Q45U18/Y67W,0.060102,0.00000,0.00000,0.025930,-0.004951,0.018609,0.041984,1.332270e-15,-0.025276,-0.007082,0.0,1.088840e-02,0.000000,0.000000,0.0,0.000000e+00


In [5]:
fp_ = 'af2.trim_bf.repairpdb.pssm.tsv.gz'
df_pssm.to_csv(fp_, header=True, index=False, sep='\t')
!md5sum {fp_}

ac6de40522b43a1e371581a5523caa96  af2.trim_bf.repairpdb.pssm.tsv.gz


In [6]:
# sstat columns for CPU time, mem usage: https://scicomp.ethz.ch/wiki/Using_the_batch_system#sstat
#df_sstat = pd.concat(map(lambda struct_id: pd.read_csv(pfile(struct_id=struct_id, step='af2.trim_bf.repairpdb.pssm', suffix='_sstat.tsv', base=''), sep='\t').assign(struct_id=struct_id), l_pssm), axis=0).query('`JobID`.str.endswith(".batch")')
#df_sstat['nresid_af2'] = df_sstat['struct_id'].map(lambda struct_id: read_chain_len(pfile(struct_id=struct_id, step='af2', suffix='.pdb', base=''), chain_='A'))
#df_sstat['nresid_trim_bf'] = df_sstat['struct_id'].map(lambda struct_id: read_chain_len(pfile(struct_id=struct_id, step='af2.trim_bf', suffix='.pdb', base=''), chain_='A'))
#dispall(df_sstat[['struct_id', 'JobID', 'MinCPU', 'MaxRSS', 'nresid_af2', 'nresid_trim_bf']].sort_values('nresid_trim_bf'))